In [5]:
import sys
import os


PROJECT_ROOT = "/home/tibia/Documents/LLM/BrainGPT/"

if PROJECT_ROOT not in sys.path:

    sys.path.append(PROJECT_ROOT)
    print(f"Ajout de la racine du projet au sys.path : {PROJECT_ROOT}")
else:
    print("La racine du projet est déjà dans le sys.path.")


Ajout de la racine du projet au sys.path : /home/tibia/Documents/LLM/BrainGPT/


## BrainGPT Pipleline

In [1]:
import sys
import os

# Définir le chemin du répertoire 'evaluation' qui contient 'flamingo'
EVALUATION_PATH = "/home/tibia/Documents/LLM/BrainGPT/evaluation/"

# 1. S'assurer que le répertoire 'evaluation/' est dans le chemin de recherche
if EVALUATION_PATH not in sys.path:
    # 2. Ajout de ce chemin pour que Python puisse résoudre 'from flamingo...'
    sys.path.append(EVALUATION_PATH)
    print(f"Ajout du chemin d'évaluation au sys.path : {EVALUATION_PATH}")

# 3. Vérification de l'importation de flamingo
try:
    import flamingo
    print("\nSUCCÈS: 'flamingo' est visible !")
    print(f"Flamingo importé depuis : {flamingo.__file__}") 
except ModuleNotFoundError:
    print("\nÉCHEC: 'flamingo' n'est toujours pas visible. Problème de configuration.")


Ajout du chemin d'évaluation au sys.path : /home/tibia/Documents/LLM/BrainGPT/evaluation/

SUCCÈS: 'flamingo' est visible !
Flamingo importé depuis : /home/tibia/Documents/LLM/BrainGPT/evaluation/flamingo/__init__.py


/home/tibia/Documents/LLM/BrainGPT/braingpt_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

from otter.modeling_otter import OtterForConditionalGeneration

model = OtterForConditionalGeneration.from_pretrained(
                "/home/tibia/Documents/LLM/BrainGPT/checkpoints/OTTER_CLIP_BRAINGPT_hf/"
)

# Affichez le modèle pour vérifier qu'il est chargé (facultatif)
print(f"Modèle chargé : {type(model)}")

tokenizer = model.text_tokenizer

[2025-12-15 17:14:59,240] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


[2025-12-15 17:14:59,559] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


/home/tibia/Documents/LLM/BrainGPT/braingpt_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


/home/tibia/Documents/LLM/BrainGPT/evaluation/flamingo/mpt/param_init_fns.py:28: UserWarning: Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:306.)
  init_fn_(module.weight[slice_indices])
Using pad_token, but it is not set yet.


The current model version is configured for Otter-Image with max_num_frames set to None.
Total Trainable param: 1.385404 B


Loading checkpoint shards: 100%|██████████| 4/4 [00:18<00:00,  4.71s/it]
Some weights of OtterForConditionalGeneration were not initialized from the model checkpoint at /home/tibia/Documents/LLM/BrainGPT/checkpoints/OTTER_CLIP_BRAINGPT_hf/ and are newly initialized: ['vision_encoder.vision_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modèle chargé : <class 'otter.modeling_otter.OtterForConditionalGeneration'>


### Création dataset : 

In [6]:
# 1. Création d'une fausse classe d'arguments pour remplacer argparse braingpt
class SimpleArgs:
    def __init__(self, tokenizer):
        # Paramètres requis par MimicitDataset.__init__
        self.tokenizer = tokenizer
        self.task = "inference"
        self.max_src_length = 512      # Valeur par défaut standard
        self.max_tgt_length = 512      # Valeur par défaut standard
        self.seed = 42                 # Valeur par défaut standard
        self.patch_image_size = 224  # Taille standard des patchs image
        self.inst_format= "simple"
        
        # Paramètres pour le DataLoader
        self.workers = 1

In [7]:
import torch
from torch.utils.data import DataLoader
from pipeline.mimicit_utils.mimicit_dataset import MimicitDataset

def get_inference_dataloader(args, tokenizer):
    
    
    """
    Version simplifiée pour l'inférence avec --mimicit_path et --images_path uniquement.
    """
    # 1. Configuration minimale requise par MimicitDataset
    args = SimpleArgs(tokenizer)
    
    # 2. Préparation des listes de chemins
    # MimicitDataset attend des listes, même pour un seul fichier
    mimicit_paths = ["/home/tibia/Documents/LLM/BrainGPT/data/instruction_dataset_3.json"]
    image_paths = ["/home/tibia/Documents/LLM/BrainGPT/data/Adri.json"]

    # Gestion de la config d'entraînement (probablement vide pour vous, mais requise par la classe)
    #  on passe une liste avec une chaîne vide
    train_config_paths = [""]
    
    # Statut des données (tout est "new" pour l'inférence)
    status_list = ["new"]

    print(f"Chargement du dataset depuis : {mimicit_paths[0]} et {image_paths[0]}")

    # 3. Création de l'instance du Dataset
    # On suppose que la classe MimicitDataset est importée
    dataset = MimicitDataset(
        args, 
        mimicit_paths, 
        image_paths, 
        train_config_paths, 
        status_list=status_list
    )

    # 4. Création du DataLoader simple
    # - batch_size=1 : Pour traiter une image à la fois
    # - shuffle=False : IMPORTANT pour l'inférence (garder l'ordre des fichiers)
    # - drop_last=False : On veut traiter TOUTES les images, même si le compte n'est pas rond
    dataloader = DataLoader(
        dataset,
        batch_size=1, 
        num_workers=args.workers, # ou 1
        pin_memory=True,
        shuffle=False, 
        drop_last=False,
        collate_fn=dataset.collate # On garde la méthode de formatage du dataset
    )

    # On retourne directement le dataloader (pas une liste de dataloaders)
    return dataloader

In [8]:
device_id = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device_id)
batch_mimicit = next(iter(get_inference_dataloader(SimpleArgs(tokenizer), tokenizer)))


Chargement du dataset depuis : /home/tibia/Documents/LLM/BrainGPT/data/instruction_dataset_3.json et /home/tibia/Documents/LLM/BrainGPT/data/Adri.json
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


####  Test sur le batch_mimicit:

In [ ]:
print("Batch keys:", batch_mimicit.keys())

# id
# On vérifie si c'est une liste avant de demander la longueur, pour être sûr
if isinstance(batch_mimicit["id"], (list, tuple)):
    print("id length:", len(batch_mimicit["id"]))
    print("Premier ID:", batch_mimicit["id"][0]) # Utile pour vérifier
else:
    print("id (valeur):", batch_mimicit["id"])

# net_input
print("net_input keys:", batch_mimicit["net_input"].keys())


# On affiche la valeur directe et le type
print("nsentences type:", type(batch_mimicit["nsentences"]))
print("nsentences value:", batch_mimicit["nsentences"])

Batch keys: dict_keys(['id', 'nsentences', 'net_input'])
id (valeur): ['INS_ID_00526c11_ID_d6296de728']
net_input keys: dict_keys(['input_ids', 'attention_masks', 'patch_images'])
nsentences type: <class 'int'>
nsentences value: 1


### Train_one_epoch

In [12]:


media_token_id = tokenizer("<image>", add_special_tokens=False)["input_ids"][-1]
endofchunk_token_id = tokenizer("<|endofchunk|>", add_special_tokens=False)["input_ids"][-1]
answer_token_id = tokenizer("<answer>", add_special_tokens=False)["input_ids"][-1]
ens_token_id = tokenizer(tokenizer.eos_token, add_special_tokens=False)["input_ids"][-1]

model.eval()


images = batch_mimicit["net_input"]["patch_images"].to(device_id, non_blocking=True)
input_ids = batch_mimicit["net_input"]["input_ids"].to(device_id, non_blocking=True)
attention_mask = batch_mimicit["net_input"]["attention_masks"].to(device_id, non_blocking=True)

labels = input_ids.clone()
labels[labels == tokenizer.pad_token_id] = -100
labels[:, 0] = -100
for i in range(labels.shape[0]):
                # remove loss for any token before the first <image> token
                # label_idx = 0
                # while label_idx < labels.shape[1] and labels[i][label_idx] != media_token_id:
                #     labels[i][label_idx] = -100
                #     label_idx += 1

                # <image>User: {cur_incontext_instruction} GPT:<answer> {cur_incontext_answer}<|endofchunk|>User: {instruction} GPT:<answer> {answer}<|endofchunk|>
                # <image>User: {cur_incontext_instruction} GPT:<answer> {cur_incontext_answer}<|endofchunk|><image>User: {instruction} GPT:<answer> {answer}<|endofchunk|>

                # get index of all endofchunk/media tokens in the sequence
    endofchunk_idxs = torch.where(labels[i] == endofchunk_token_id)[0]
    media_idxs = torch.where(labels[i] == media_token_id)[0]

                # remove loss for any token the before the first <answer>
    token_idx = 0
    while token_idx < labels.shape[1] and labels[i][token_idx] != answer_token_id:
        labels[i][token_idx] = -100
        token_idx += 1

                # remove loss for any token between <|endofchunk|> and <answer>, except <image>
        for endofchunk_idx in endofchunk_idxs[:-1]:
            token_idx = endofchunk_idx + 1
            while token_idx < labels.shape[1] and labels[i][token_idx] != answer_token_id:
                if labels[i][token_idx] == media_token_id:
                    pass
                else:
                    labels[i][token_idx] = -100
                    token_idx += 1

labels[labels == answer_token_id] = -100
labels[labels == media_token_id] = -100

In [18]:
# images ??




device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
dtype = model.dtype

test_prompt = "Describe the image in a single sentence."
lang_x = model.text_tokenizer(
                    [f"<image>User: {test_prompt} GPT:<answer>"],
                    return_tensors="pt",
                )
print(f" lang x : {lang_x}")

lang_x_input_ids = lang_x["input_ids"].to(device)
lang_x_attention_mask = lang_x["attention_mask"].to(device)



generated_text = model.generate(
                    vision_x=images.to(dtype),
                    lang_x=lang_x_input_ids,
                    attention_mask=lang_x_attention_mask,
                    max_new_tokens = 512,
                ) 

Setting `pad_token_id` to `eos_token_id`:50277 for open-end generation.


 lang x : {'input_ids': tensor([[50278,  6989,    27,  3666, 19268,   253,  2460,   275,   247,  2014,
          6197,    15,   443,  5736,    27, 50279]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [19]:
# géneration du texte 
import pandas as pd

generated_captions = {}


parsed_output = (
                    model.text_tokenizer.decode(generated_text[0])
                    .split("<answer>")[-1]
                    .lstrip()
                    .rstrip()
                    .split("<|endofchunk|>")[0]
                    .lstrip()
                    .rstrip()
                    .lstrip('"')
                    .rstrip('"')
                )
gt = (
                    model.text_tokenizer.decode(input_ids[0])
                    .split("<answer>")[-1]
                    .lstrip()
                    .rstrip()
                    .split("<|endofchunk|>")[0]
                    .lstrip()
                    .rstrip()
                    .lstrip('"')
                    .rstrip('"')
                )
                # print(batch_mimicit.keys())
#                 print("/",parsed_output,"/")
generated_captions[batch_mimicit["id"][0]] = (gt, parsed_output)
                # print(generated_captions.keys())

    # print(generated_captions)
df_data = [(key, val[0], val[1]) for key, val in generated_captions.items()]
df = pd.DataFrame(df_data, columns=['id', 'gt', 'parsed_output'])
print(df)

df.to_csv("/home/tibia/Documents/LLM/BrainGPT/notebooks/output2", index=False)

                              id gt  \
0  INS_ID_00526c11_ID_d6296de728      

                                       parsed_output  
0  > Mild generalized atrophic change of the brai...  


### Test tokenizer

In [ ]:
## Attriut du modèle pour vérifier qu'il est bien chargé
print(f"Nombre de paramètres du modèle : {sum(p.numel() for p in model.parameters())}")
print(f"tokenizer du modèle : {model.text_tokenizer} ")


# analyse tokenizer 

tokenizer= model.text_tokenizer

print("--- Type du Tokenizer ---")
print(type(tokenizer))


print("\n--- Configuration et Vocabulaire ---")
print(f"Taille du vocabulaire : {len(tokenizer)}")
print(f"Token de fin de séquence (EOS) : {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
print(f"Token de padding (PAD) : {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")

# 3. Test simple  ?
test_phrase = "how otter deal with phrases i give him ?"
print(f"\n--- Test de Tokenisation de : '{test_phrase}' ---")
tokens = tokenizer(test_phrase, return_tensors='pt')
print (f"Tokens : {tokens}")
print(f"Tokens ID : {tokens['input_ids'][0].tolist()}") # O pour déballerr tensor
print(f"Tokens décodés : {tokenizer.convert_ids_to_tokens(tokens['input_ids'][0].tolist())}")


#test decodage 
tokenizer.decode(tokens["input_ids"][0])

## Hugging face

In [5]:
sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)



print(tokens)
print(ids)

['Using', 'Ġa', 'ĠTrans', 'former', 'Ġnetwork', 'Ġis', 'Ġsimple']
[11888, 247, 4480, 19946, 2990, 310, 2969]


In [9]:
from transformers import AutoTokenizer

tokenizer_2 = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens_2 = tokenizer_2.tokenize(sequence)

print(tokens)
ids = tokenizer_2.convert_tokens_to_ids(tokens_2)

print(ids)
decoded_string = tokenizer_2.decode(ids)
print(decoded_string)


['Using', 'Ġa', 'ĠTrans', 'former', 'Ġnetwork', 'Ġis', 'Ġsimple']
[7993, 170, 13809, 23763, 2443, 1110, 3014]
Using a Transformer network is simple


/home/tibia/Documents/LLM/BrainGPT/braingpt_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."
sequence_2 = "I hate this so much "

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

tokens_2 = tokenizer.tokenize(sequence_2)
ids_2 = tokenizer.convert_tokens_to_ids(tokens_2)



input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)
input_ids_2 = torch.tensor([ids_2])
print("Input IDs 2:", input_ids_2)

output = model(input_ids)
print("Logits:", output.logits)

output_2 = model(input_ids_2)
print("Logits 2:", output_2.logits)




pad_length = len(ids) - len(ids_2)
# Crée une liste contenant 'pad_token_id' répété 'pad_length' fois
padding_tokens = [tokenizer.pad_token_id] * pad_length

mask_1 = [1] * len(ids) 
mask_2 = ([1] * len(ids_2)) + ([0] * pad_length) 
attention_mask = [mask_1, mask_2]

batched_ids = [ids, ids_2 + padding_tokens]
input_ids = torch.tensor(batched_ids)

print("Batched Input IDs:", input_ids)
output = model(input_ids, attention_mask=torch.tensor([attention_mask]))
print("Batched Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Input IDs 2: tensor([[1045, 5223, 2023, 2061, 2172]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)
Logits 2: tensor([[ 3.1744, -2.6848]], grad_fn=<AddmmBackward0>)
Batched Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012],
        [ 1045,  5223,  2023,  2061,  2172,     0,     0,     0,     0,     0,
             0,     0,     0,     0]])
Batched Logits: tensor([[-2.7276,  2.8789],
        [ 3.1744, -2.6848]], grad_fn=<AddmmBackward0>)


In [14]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]
# problème ici 
print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)


attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

/home/tibia/Documents/LLM/BrainGPT/braingpt_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. Charger le modèle et le tokenizer
model_name = "gpt2" # Exemple
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 2. Préparer l'entrée
prompt = "Hugging Face est une entreprise qui promeut"
inputs = tokenizer(prompt, return_tensors="pt") # Convertir le texte en tenseurs

# 3. Générer le texte
generated_ids = model.generate(
    **inputs,
    max_new_tokens=50,       # Max 50 nouveaux jetons
    do_sample=True,          # Utiliser l'échantillonnage
    top_p=0.95,              # Filtrer avec Top-P
    temperature=0.7,         # Contrôler l'aléa
    repetition_penalty=1.2,  # Éviter la répétition
)

# 4. Décoder le résultat
output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(output)

/home/tibia/Documents/LLM/BrainGPT/braingpt_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tibia/Documents/LLM/BrainGPT/braingpt_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[2025-12-15 12:11:07,253] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


[2025-12-15 12:11:07,577] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


/home/tibia/Documents/LLM/BrainGPT/braingpt_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hugging Face est une entreprise qui promeut le nouvelles, les corsages du fait sur deux.
(1) In this passage there is a slight difference between the French and English translations of "The Little Ghost" (1704), where one character has its


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)
print("f shape en sortie du modèle :", output.logits.shape)  # Devrait être (2, 2) pour deux séquences et deux classes

/home/tibia/Documents/LLM/BrainGPT/braingpt_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


f shape en sortie du modèle : torch.Size([2, 2])


In [10]:
from transformers import CLIPImageProcessor

image_processor = CLIPImageProcessor()
print("--- Type du Processeur d'Image ---")
print(type(image_processor))

# 2. Afficher les paramètres de prétraitement (très importants)
print("\n--- Paramètres de Prétraitement ---")
print(f"Taille de redimensionnement (resize) : {image_processor.size}")
print(f"Taille de recadrage (crop size) : {image_processor.crop_size}")
print(f"Moyenne (mean) utilisée pour la normalisation : {image_processor.image_mean}")
print(f"Écart-type (std) utilisé pour la normalisation : {image_processor.image_std}")

--- Type du Processeur d'Image ---
<class 'transformers.models.clip.image_processing_clip.CLIPImageProcessor'>

--- Paramètres de Prétraitement ---
Taille de redimensionnement (resize) : {'shortest_edge': 224}
Taille de recadrage (crop size) : {'height': 224, 'width': 224}
Moyenne (mean) utilisée pour la normalisation : [0.48145466, 0.4578275, 0.40821073]
Écart-type (std) utilisé pour la normalisation : [0.26862954, 0.26130258, 0.27577711]


In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

encoded_input = tokenizer("Hello, I'm a single sentence!")
print(encoded_input)


tokenized_text = "Jim Henson was a puppeteer".split()
print(tokenized_text)

{'input_ids': [101, 8667, 117, 146, 112, 182, 170, 1423, 5650, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['Jim', 'Henson', 'was', 'a', 'puppeteer']
